# ipl I317B Sécurité : labos
## Semaine 1 : introduction et outils python

### Partie 1 : manipuler du texte

#### Exercice 1
Pour ce premier exercice, nous allons manipuler un court extrait de logs d'un serveur web nginx.  
Pour chaque ligne de celui-ci, extrayez:
- l'adresse IP
- le timestamp
- le chemin demandé

In [1]:
logs = """91.160.48.68 - - [3/Sep/2020:18:18:10 +0200] "GET /?C=M;O=D HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0" "-"
78.224.72.158 - - [3/Sep/2020:18:24:52 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0" "-"
192.241.199.4 - - [3/Sep/2020:19:11:15 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 zgrab/0.x" "-"
40.121.69.161 - - [3/Sep/2020:19:16:32 +0200] "GET /.env HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" "-"
40.121.69.161 - - [3/Sep/2020:19:16:32 +0200] "POST / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" "-"
185.172.110.223 - - [3/Sep/2020:19:16:37 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0" "-"
66.249.75.16 - - [3/Sep/2020:19:23:28 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" "-"
66.249.75.23 - - [3/Sep/2020:19:23:30 +0200] "GET / HTTP/1.1" 200 0 "-" "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" "-" """

In [2]:
for line in logs.split("\n"):
    print("ip:",   line.split(" ", 1)[0])
    print("ts:",   line.split("[", 1)[1].split("]")[0])
    print("path:", line.split('"')[1].split()[1])
    print("-" * 10)

ip: 91.160.48.68
ts: 3/Sep/2020:18:18:10 +0200
path: /?C=M;O=D
----------
ip: 78.224.72.158
ts: 3/Sep/2020:18:24:52 +0200
path: /
----------
ip: 192.241.199.4
ts: 3/Sep/2020:19:11:15 +0200
path: /
----------
ip: 40.121.69.161
ts: 3/Sep/2020:19:16:32 +0200
path: /.env
----------
ip: 40.121.69.161
ts: 3/Sep/2020:19:16:32 +0200
path: /
----------
ip: 185.172.110.223
ts: 3/Sep/2020:19:16:37 +0200
path: /
----------
ip: 66.249.75.16
ts: 3/Sep/2020:19:23:28 +0200
path: /
----------
ip: 66.249.75.23
ts: 3/Sep/2020:19:23:30 +0200
path: /
----------


#### Exercice 2
Avec les mêmes logs que l'exercice précédent, comptez et affichez les différents user-agent.

In [16]:
all_ua = {}

for line in logs.split("\n"):
    ua = line.split('"')[5]
    
    if ua not in all_ua:
        all_ua[ua] = 0

    all_ua[ua] += 1
    
for k, v in all_ua.items():
    print(v, "=>", k)

2 => Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0
1 => Mozilla/5.0 zgrab/0.x
2 => Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36
1 => Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0
2 => Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)


### Partie 2 : requests et BeautifulSoup

Assurez vous que les paquets suivant soient bien installé :

`pip3 install --user requests bs4`

`sudo pip3 install requests bs4`

Depuis un jupyter notebook, vous pouvez probablement juste executer la case suivante :

In [57]:
!pip3 install --user requests bs4

Le packet requests nous permet de faire des requêtes web facilement, par exemple :

In [4]:
import requests

ipl_req = requests.get("https://www.vinci.be/fr/formations/informatique-de-gestion")

if ipl_req.ok:
    ipl_html = ipl_req.text
else:
    print(ipl_req.status_code)
    print(ipl_req.reason)

# Doc requests : https://requests.readthedocs.io/en/latest/ )

Nous pouvons ensuite lire le code source de la page en affichant `ipl_html`.

Parser du html avec des split ou des regex deviendrait vite infernal, c'est là qu'intervient la librairie BeautifulSoup qui nous permet de parcourir beaucoup plus simplement le dom notamment avec les fonctions `find` et `find_all`.

In [5]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(ipl_html)

# pour récupérer le contenu de la balise <title> :
title = soup.find("title").get_text()

print(title)

Informatique de gestion | HE VINCI


La fonction `find` et la fonction `find_all` vous renverons un object de type `tag` (balise) sur lequel vous pouvez faire :
  * d'autres `find()` et `find_all()`
  * `get_text()` pour récupérer tout le contenu textuel de la balise
  * `["nom de l'attribut"]` pour récupérer un attribut en particulier (exemple: `tag['href']`)
  * ...

( Doc BeautifulSoup4 : https://www.crummy.com/software/BeautifulSoup/bs4/doc/ )

#### Exercice 1:
Récupérez le contenue de la page suivante et affichez son titre proprement : https://www.vinci.be/fr/formations/informatique-de-gestion

In [6]:
from bs4 import BeautifulSoup
import requests

ipl_req = requests.get("https://www.vinci.be/fr/formations/informatique-de-gestion")
ipl_html = ipl_req.text

soup = BeautifulSoup(ipl_html)

# pour récupérer le contenu de la balise <title> :
title = soup.find("title").get_text()

print(title)

Informatique de gestion | HE VINCI


Il est possible de faire des requêtes beaucoup plus poussée en ajoutant des arguments correspondant aux attributs des balises que nous cherchons, par exemple pour trouver les balises `<div class="footer"></div>` :

```
soup.find_all("div", class_="footer")
# NB: class est un mot clé réservé en python, d'où l'ajout du "_" dans BeautifulSoup
```

Ou encore pour trouver tous les liens vers la page facebook de l'ipl :

`soup.find_all("a", href="https://www.facebook.com/HEVINCI")`

BeautifoulSoup accepte également des regex compilée en argument, il est donc possible de récupérer la liste des url qui ne sont pas du domaines de "vinci.be" :

```
import re

not_vinci = re.compile("^http(s?)://(?!www.vinci.be/).*$")
soup.find_all("a", href=not_vinci)
```

Il est également possible d'utiliser une fonction arbitraire pour valider les liens. Par exemple, pour trouver tous les liens de moins de 30 caractères :

```
def less_than_30(tag):
    return len(tag) < 30  # on retourne un booleen
    
# on passe notre fonction en paramètre sans l'executer :
soup.find_all("a", href=less_than_30)
```

#### Exercice 2:

Utilisez des regex pour récupérer la liste des liens http (et donc pas https) sur la page :  
https://www.vinci.be/fr/formations/informatique-de-gestion

In [7]:
ipl_https = requests.get("https://www.vinci.be/fr/formations/informatique-de-gestion")

In [8]:
import re

not_https = re.compile("^http://.*$")

for link in soup.find_all("a", href=not_https):
    print(link['href'])

http://www.enseignement.be/index.php?page=4323


In [9]:
import re

def not_https(href):
    return href.startswith("http://")

for link in soup.find_all("a", href=not_https):
    print(link['href'])

http://www.enseignement.be/index.php?page=4323


#### Exercice 3:

Utilisez une fonction pour récupérer tous les liens qui finissent avec une extension désignant un type de ficher (ex: ".html", ".php", ".pdf", ...) sur la page :  
https://www.vinci.be/fr/formations/informatique-de-gestion

In [10]:
import re

ext = re.compile("^https://.*/.*\.[a-zA-Z]{1,5}$")

for link in soup.find_all("a", href=ext):
    print(link['href'])

https://www.vinci.be/uploads/brochures/INFORMATIQUE_GESTION_HEVINCI_21-22.pdf
https://progcours.vinci.be/cocoon/programmes/E1BINV01_C.html


#### Exercice 4:
Depuis le site du Centre Cybersecurité Belgique https://ccb.belgium.be/fr/actualit%C3%A9s , récupérez la liste des articles récents et affichez leurs titres ainsi qu'un lien vers l'article.

In [25]:
page = requests.get("https://ccb.belgium.be/fr/actualit%C3%A9s")
soup = BeautifulSoup(page.text)

for article in soup.find_all("article"):
    print(article.find("h2").text)
    print("https://ccb.belgium.be" + article["onclick"].split("'")[1])
    
    print("---")

10 talents belges ont participé à la compétition European Cybersecurity Challenge
https://ccb.belgium.be/fr/actualit%C3%A9/10-talents-belges-ont-particip%C3%A9-%C3%A0-la-comp%C3%A9tition-european-cybersecurity-challenge
---
Etape importante dans la certification de cybersécurité : un cadre juridique pour la délivrance et la reconnaissance de certificats européens liés à la cybersécurité est publié
https://ccb.belgium.be/fr/actualit%C3%A9/etape-importante-dans-la-certification-de-cybers%C3%A9curit%C3%A9-un-cadre-juridique-pour-la
---
NCC-BE session d'information: EU Cybersecurity Funding Opportunities 
https://ccb.belgium.be/fr/actualit%C3%A9/ncc-be-session-dinformation-eu-cybersecurity-funding-opportunities
---
Nouveau guide pratique de cybersécurité pour les écoles publié par la Fédération Wallonie-Bruxelles
https://ccb.belgium.be/fr/actualit%C3%A9/nouveau-guide-pratique-de-cybers%C3%A9curit%C3%A9-pour-les-%C3%A9coles-publi%C3%A9-par-la-f%C3%A9d%C3%A9ration-wallonie
---
Cyber Europe 2

#### Exercice 5:
La lib requests permet également d'effectuer de requête POST. Pour tester cette fonctionnalitée, créez un "requestbin" sur le site http://requestbin.net/ puis utilisez requests pour poster des données vers votre requestbin.

> RequestBin gives you a URL that will collect requests made to it and let you inspect them in a human-friendly way.
Use RequestBin to see what your HTTP client is sending or to inspect and debug webhook requests.


Voir : https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request (ou la cheatsheet)

In [27]:
import requests, time

# https://requestbin.net/bins/view/4c37852bd009ba9399129d3f479e5f47ea131242

r = requests.post('http://5mj29klz7ltbctx2.b.requestbin.net', data={"ts":time.time()})

print(r.status_code)
print(r.content)

200
b'16636289691644104\n'


#### Exercice 6:

Maintenant que vous maîtrisez le get, le post et quelques bases de parsing web, "connectez-vous" au formulaire suivant avec les identifiants : `admin/admin`  
http://labosecuipl.alwaysdata.net/21/s01/form/

In [ ]:
req = requests.post("http://labosecuipl.alwaysdata.net/21/s01/form/", data={
    "login": "admin",
    "user": "admin"
})

print(req.text)

<!DOCTYPE html>
<html lang="en">
<p id='result'>Bad login !<p>
<form method='POST'>
  Login:<br>
  <input type='text' name='login'><br>
  Password:<br>
  <input type='password' name='password'><br>
  <input type='hidden' name='token' value='pjpge621xxc00g0csk0008gswc44k4k'>
  <input type='submit' value='Submit'>
</form></html>



In [33]:
session = requests.Session()

# fetch the "csrf" token
req = session.get("http://labosecuipl.alwaysdata.net/21/s01/form/")
soup = BeautifulSoup(req.text)
token = soup.find('input', {'name': 'token'})["value"]

# connect
req = session.post("http://labosecuipl.alwaysdata.net/21/s01/form/", data={
    "login": "admin",
    "password": "admin",
    "token": token
})

print(req.text)

<!DOCTYPE html>
<html lang="en">
<p id='result'>Connected !</p></html>



#### Exercice 7:
Vous avez certainement constaté sur votre requestbin que la librairie requests utilise par défaut un User-Agent explicite "*je suis un lib python pour faire des requêtes*".  
Documentez-vous sur la manière de changer celui-ci dans requests et remplacez le dans votre précédente requête vers requestbin par quelque chose de plus discret, par exemple l'user-agent de la dernière version de chrome :  
`Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36`

source: https://www.whatismybrowser.com/guides/the-latest-user-agent/chrome

In [47]:
req = requests.get("http://whatsmyuseragent.org/")

if req.status_code != 200:
    print("Error: %s\nReason: %s" % (req.status_code, req.reason))
else:
    soup = BeautifulSoup(req.text)

    soup.find("div", {"class": "user-agent"}).text.strip()

Error: 406
Reason: Not Acceptable


In [48]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36',
}

req = requests.get("http://whatsmyuseragent.org/", headers=headers)

soup = BeautifulSoup(req.text)
soup.find("div", {"class": "user-agent"}).text.strip()

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'

# Partie 3 (bonus):

Note: dans les jupyter notebook, vous pouvez exécuter des commandes shell en préfixant une cellule d'un `!`. Par exemple : `!date`.


Exercice 1

Pour ce premier exercice, nous allons manipuler un court extrait de logs "logs.txt" d'un serveur web nginx (le même que pour la partie 1).

Pour chaque ligne de celui-ci, extrayez:

    l'adresse IP
    le timestamp
    le chemin demandé

En n'utilisant que des commandes bash : cat, grep, cut, sort, uniq,...

In [51]:
!cat logs.txt

91.160.48.68 - - [3/Sep/2020:18:18:10 +0200] "GET /?C=M;O=D HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0" "-"
78.224.72.158 - - [3/Sep/2020:18:24:52 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0" "-"
192.241.199.4 - - [3/Sep/2020:19:11:15 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 zgrab/0.x" "-"
40.121.69.161 - - [3/Sep/2020:19:16:32 +0200] "GET /.env HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" "-"
40.121.69.161 - - [3/Sep/2020:19:16:32 +0200] "POST / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" "-"
185.172.110.223 - - [3/Sep/2020:19:16:37 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0" "-"
66.249.75.16 - - [3/Sep/2020:19

In [50]:
!cat logs.txt | cut -d " " -f 1

91.160.48.68
78.224.72.158
192.241.199.4
40.121.69.161
40.121.69.161
185.172.110.223
66.249.75.16
66.249.75.23


In [52]:
!cat logs.txt | cut -d '[' -f 2 | cut -d ']' -f 1

3/Sep/2020:18:18:10 +0200
3/Sep/2020:18:24:52 +0200
3/Sep/2020:19:11:15 +0200
3/Sep/2020:19:16:32 +0200
3/Sep/2020:19:16:32 +0200
3/Sep/2020:19:16:37 +0200
3/Sep/2020:19:23:28 +0200
3/Sep/2020:19:23:30 +0200


In [54]:
!cat logs.txt | cut -d ' ' -f 7

/?C=M;O=D
/
/
/.env
/
/
/
/


#### Exercice 2
Avec les mêmes logs que l'exercice précédent, comptez et affichez les différents user-agent.

In [56]:
!cat logs.txt | cut -d '"' -f 6 | sort | uniq -c

      2 Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)
      2 Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0
      2 Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36
      1 Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0
      1 Mozilla/5.0 zgrab/0.x
